In [8]:
! pip install pdfplumber


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\jimmy\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [51]:
!pip install pymongo


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\jimmy\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [88]:
import re

from PyPDF2 import PdfReader

from transformers import GPT2TokenizerFast
 
# Load tokenizer to count tokens

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
 
# Load PDF

doc_path = r"C:\Users\jimmy\Downloads\N1_Trunk_Infra_Agreement1.pdf"

reader = PdfReader(doc_path)
 
# Extract all text

full_text = ""

for page in reader.pages:

    full_text += page.extract_text() + "\n"
 
# Define section patterns

patterns = {

    "Agreement Metadata": r"(?is)(agreement.*?value:.*?)(?=Item No.|Scope|Staff|Personnel|Equipment|Notes|Technical)",

    "Scope of Work": r"(?is)(scope of work.*?)(?=Item No.|Staff|Personnel|Equipment|Notes|Technical)",

    "Staffing/Personnel Requirements": r"(?is)(Item No.*?Manager.*?)(?=Technical Requirements|Equipment|Notes|conditions)",

    "Equipment Requirements": r"(?is)(Technical Requirements.*?)(?=Chief Engineer|Notes|conditions)",

    "Notes and Conditions": r"(?is)(Note:.*)"

}
 
# Function to split long sections into chunks

def chunk_section(text, section_name, max_tokens=2000, small_chunk=800, overlap=100, metadata=None):

    tokens = tokenizer.encode(text)

    chunks = []
 
    if len(tokens) <= max_tokens:

        chunks.append({"section": section_name, "metadata": metadata or {}, "content": text.strip()})

    else:

        start = 0

        while start < len(tokens):

            end = min(start + small_chunk, len(tokens))

            chunk_text = tokenizer.decode(tokens[start:end])

            chunks.append({"section": section_name, "metadata": metadata or {}, "content": chunk_text.strip()})

            start += small_chunk - overlap

    return chunks
 
all_chunks = []
 
for name, pattern in patterns.items():
    match = re.search(pattern, full_text, re.DOTALL)
    if match:
        section_text = match.group().strip()

        if name == "Staffing/Personnel Requirements":
            rows = re.findall(r"\d+\.\s*(.*?)\s*-.*", section_text)
            for row in rows:
                metadata = {"role": row.strip()}
                section_chunks = chunk_section(section_text, name, metadata=metadata)
                all_chunks.extend(section_chunks)

        elif name == "Equipment Requirements":
            items = re.findall(r"\d+\.\s*([A-Za-z0-9\- ]+)-", section_text)
            for item in items:
                metadata = {"item": item.strip()}
                section_chunks = chunk_section(section_text, name, metadata=metadata)
                all_chunks.extend(section_chunks)

        else:
            section_chunks = chunk_section(section_text, name)
            all_chunks.extend(section_chunks)
    else:
        all_chunks.append({"section": name, "metadata": {}, "content": "<Not Found>"})
 
# Show enriched chunks

for c in all_chunks[:10]:  # preview first 10

    print("\n=====", c["section"], "=====")

    print("Metadata:", c["metadata"])

    print(c["content"][:30], "...\n")

 

Token indices sequence length is longer than the specified maximum sequence length for this model (1140 > 1024). Running this sequence through the model will result in indexing errors



===== Agreement Metadata =====
Metadata: {}
AGREEMENT
GOVERNMENT OF ANDHRA ...


===== Scope of Work =====
Metadata: {}
scope of works. The Bidder sha ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Project Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Transportation Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Structural Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Manager – QS & Billing'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Manager – QA & QC / Materials'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Construction Manager'}
Item No. | Position/specializa ...


===== Staffing/Personnel Requirements =====
Metadata: {'role': 'Manager – Construction safet

In [89]:
import json

# Save all_chunks to a JSON file
output_path = r"C:\Users\jimmy\Downloads\N1_Trunk_Infra_Agreement_chunks.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=2)

print(f"Chunks saved to {output_path}")

Chunks saved to C:\Users\jimmy\Downloads\N1_Trunk_Infra_Agreement_chunks.json


In [90]:
%pip install openai
%pip install tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [91]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os
import tiktoken
import concurrent.futures
import logging

load_dotenv("../injestion/config/.env")
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# Helper functions for embeddings and similarity
def get_openai_embedding(text, timeout=15):
    """Get embeddings using Azure OpenAI's text-embedding model with context window truncation and timeout."""
    # Truncate text to fit within model context window (e.g., 8000 tokens for text-embedding-3-small)
    max_tokens = 8000
    encoding = tiktoken.encoding_for_model("text-embedding-3-small")
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        text = encoding.decode(tokens)
    def call():
        return client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        ).data[0].embedding
    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(call)
        try:
            return future.result(timeout=timeout)
        except concurrent.futures.TimeoutError:
            logging.error(f"OpenAI embedding call timed out for text: {text[:50]}")
            raise TimeoutError("OpenAI embedding call timed out.")

In [92]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [93]:
import sys
import os
from datetime import datetime

# Add the injestion directory to Python path
sys.path.append(os.path.abspath("../injestion"))

from models.datamodel_pdantic import Module, KnowledgeObject, Chunk, EmbeddingMeta, Metadata
from scripts.mongo_utils import (
    insert_module, 
    insert_knowledge_object, 
    insert_chunk, 
    create_embedding_meta
)

# 1. Create and insert Module
module_data = {
    "module_id": "N1_Trunk_Infra_Agreement",
    "module_tag": ["infrastructure", "construction", "amaravati", "roads"],
    "module_link": ["https://example.com/n1-trunk-agreement"]
}

try:
    module_obj = Module(**module_data)
    module_id = insert_module(module_obj)  # This returns the MongoDB ObjectId as string
    print(f"✅ Module created with ID: {module_id}")
except Exception as e:
    print(f"❌ Module creation failed: {e}")
    module_id = "N1_Trunk_Infra_Agreement"  # fallback

# 2. Create Metadata for Knowledge Object
metadata = Metadata(
    path="/documents/N1_Trunk_Infra_Agreement.pdf",
    repo_url="https://github.com/your-repo/documents",
    intent_category="construction_agreement",
    version="1.0",
    modified_time=datetime.utcnow(),
    csp="amaravati_development"
)

# 3. Create and insert Knowledge Object
knowledge_object_data = {
    "title": "N1 Trunk Infrastructure Agreement",
    "named_entity": "BSR Infratech India Limited",
    "summary": "Construction agreement for smart trunk infrastructure in Amaravati Capital City including roads, storm water drains, water supply network, sewerage network, utility ducts, and other infrastructure components.",
    "content": "Agreement for construction of balance smart trunk infrastructure with Roads, Storm Water Drains, Water Supply Network, Sewerage Network, Utility Ducts for Power& ICT, Reuse waterline, Pedestrian tracks, Cycle tracks, Avenue Plantation& Street Furniture etc. in N1 Road in Amaravati Capital City",
    "keywords": "infrastructure, construction, amaravati, roads, drainage, utilities, BSR Infratech",
    "texts": " ".join([chunk["content"] for chunk in all_chunks if chunk["content"] != "<Not Found>"]),
    "is_terraform": False,
    "metadata": metadata,
    "module_id": module_id,  # Use the actual ObjectId as string
    "chunk_ids": []
}

try:
    knowledge_obj = KnowledgeObject(**knowledge_object_data)
    insert_knowledge_object(knowledge_obj)
    print("✅ Knowledge object created successfully")
except Exception as e:
    print(f"❌ Knowledge object creation failed: {e}")

# 4. Process and insert Chunks with embeddings
chunk_ids = []

for i, chunk_dict in enumerate(all_chunks):
    try:
        # Skip chunks with no content
        if chunk_dict["content"] == "<Not Found>" or not chunk_dict["content"].strip():
            print(f"⏭️ Skipping empty chunk {i+1}")
            continue
            
        # Generate embedding for the chunk content
        print(f"🔄 Generating embedding for chunk {i+1}/{len(all_chunks)}...")
        embedding = get_openai_embedding(chunk_dict["content"])
        
        # Create embedding metadata
        embedding_meta = create_embedding_meta(
            model_name="text-embedding-3-small",
            dimensionality=len(embedding),
            embedding_method="azure_openai"
        )
        
        # Prepare chunk data for Pydantic model (matching the expected schema)
        chunk_data = {
            "document_id": module_id,  # Use module_id (ObjectId as string) for document_id
            "chunk_id": i,
            "chunk_start": 0,  # You can calculate actual start positions if needed
            "chunk_end": len(chunk_dict["content"]),
            "chunk_text": chunk_dict["content"],
            "embedding": embedding,
            "embedding_meta": embedding_meta
        }
        
        # Create Chunk object and insert
        chunk_obj = Chunk(**chunk_data)
        chunk_id = insert_chunk(chunk_obj)
        
        if chunk_id:
            chunk_ids.append(chunk_id)
            print(f"✅ Inserted chunk {i+1}/{len(all_chunks)}: {chunk_dict['section'][:50]}...")
        else:
            print(f"❌ Failed to insert chunk {i+1}")
            
    except Exception as e:
        print(f"❌ Error processing chunk {i+1}: {e}")
        continue

print(f"\n🎉 Successfully processed {len(chunk_ids)} out of {len(all_chunks)} chunks")

# 5. Update Knowledge Object with chunk IDs
try:
    from scripts.mongo_utils import update_knowledge_object_chunk_ids
    update_knowledge_object_chunk_ids(
        module_id=module_id,
        title=knowledge_object_data["title"],
        chunk_ids=chunk_ids
    )
    print("✅ Updated knowledge object with chunk IDs")
except Exception as e:
    print(f"❌ Failed to update knowledge object with chunk IDs: {e}")

print(f"\n📊 Summary:")
print(f"  - Module: {module_id}")
print(f"  - Knowledge Object: {knowledge_object_data['title']}")
print(f"  - Chunks processed: {len(chunk_ids)}")
print(f"  - MongoDB collections updated: modules, knowledge_objects, chunks")

✅ Inserted module: 68cbf1e48bc9122af55fc36f
✅ Module created with ID: 68cbf1e48bc9122af55fc36f


C:\Users\jimmy\AppData\Local\Temp\ipykernel_27116\2720541164.py:37: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  modified_time=datetime.utcnow(),


✅ Upserted knowledge object for module: 68cbf1e48bc9122af55fc36f
✅ Knowledge object created successfully
🔄 Generating embedding for chunk 1/29...
✅ Inserted chunk: 68cbf1e68bc9122af55fc370
✅ Inserted chunk 1/29: Agreement Metadata...
🔄 Generating embedding for chunk 2/29...
✅ Inserted chunk: 68cbf1e68bc9122af55fc371
✅ Inserted chunk 2/29: Scope of Work...
🔄 Generating embedding for chunk 3/29...
✅ Inserted chunk: 68cbf1e78bc9122af55fc372
✅ Inserted chunk 3/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 4/29...
✅ Inserted chunk: 68cbf1e78bc9122af55fc373
✅ Inserted chunk 4/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 5/29...
✅ Inserted chunk: 68cbf1e88bc9122af55fc374
✅ Inserted chunk 5/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 6/29...
✅ Inserted chunk: 68cbf1e88bc9122af55fc375
✅ Inserted chunk 6/29: Staffing/Personnel Requirements...
🔄 Generating embedding for chunk 7/29...
✅ Inserted chunk: 68cbf1e88bc9122